In [1]:
# 한 셀에서 여려 결과 출력
from IPython.core.interactiveshell import InteractiveShell
InteractiveShell.ast_node_interactivity="all"

In [3]:
import pandas as pd
df = pd.read_csv("./data/0307/data.csv")
df.head()

,Unnamed: 0.1,Desc,Unnamed: 0,author,genre,image_link,rating,title
0,0,We know that power is shifting: From West to E...,0.0,Moisés Naím,Business,https://i.gr-assets.com/images/S/compressed.ph...,3.63,The End of Power: From Boardrooms to Battlefie...
1,1,Following the success of The Accidental Billio...,1.0,Blake J. Harris,Business,https://i.gr-assets.com/images/S/compressed.ph...,3.94,"Console Wars: Sega, Nintendo, and the Battle t..."
2,2,How to tap the power of social software and ne...,2.0,Chris Brogan,Business,https://i.gr-assets.com/images/S/compressed.ph...,3.78,Trust Agents: Using the Web to Build Influence...
3,3,William J. Bernstein is an American financial ...,3.0,William J. Bernstein,Business,https://i.gr-assets.com/images/S/compressed.ph...,4.20,The Four Pillars of Investing
4,4,Amazing book. And I joined Steve Jobs and many...,4.0,Akio Morita,Business,https://i.gr-assets.com/images/S/compressed.ph...,4.05,Made in Japan: Akio Morita and Sony


In [4]:
from nltk.tokenize import RegexpTokenizer
from nltk.corpus import stopwords
import re

In [5]:
# 아스키 문자 아니면 제거  
def _removeNonAscii(s):
    return "".join(i for i in s if  ord(i)<128) # chr(128) : '\x80'

def make_lower_case(text):
    return text.lower()

def remove_stop_words(text):
    text = text.split()
    stops = set(stopwords.words("english"))
    text = [w for w in text if not w in stops]
    text = " ".join(text)
    return text

def remove_html(text):
    html_pattern = re.compile('<.*?>')
    return html_pattern.sub(r'', text)

def remove_punctuation(text):
    tokenizer = RegexpTokenizer(r'[a-zA-Z]+')
    text = tokenizer.tokenize(text)
    text = " ".join(text)
    return text

df['cleaned'] = df['Desc'].apply(_removeNonAscii)
df['cleaned'] = df.cleaned.apply(make_lower_case)
df['cleaned'] = df.cleaned.apply(remove_stop_words)
df['cleaned'] = df.cleaned.apply(remove_punctuation)
df['cleaned'] = df.cleaned.apply(remove_html)


In [6]:
# 위에서 생성한 함수
#def make_lower_case(text):
#    return text.lower()
df["title_lower"] = df["title"].apply(make_lower_case)

df.head(1)

,Unnamed: 0.1,Desc,Unnamed: 0,author,genre,image_link,rating,title,cleaned,title_lower
0,0,We know that power is shifting: From West to E...,0.0,Moisés Naím,Business,https://i.gr-assets.com/images/S/compressed.ph...,3.63,The End of Power: From Boardrooms to Battlefie...,know power shifting west east north south pres...,the end of power: from boardrooms to battlefie...


In [7]:
df['title_lower'][:5]

0    the end of power: from boardrooms to battlefie...
1    console wars: sega, nintendo, and the battle t...
2    trust agents: using the web to build influence...
3                        the four pillars of investing
4                  made in japan: akio morita and sony
Name: title_lower, dtype: object

In [8]:
from gensim.models.doc2vec import TaggedDocument
from konlpy.tag import Okt
okt = Okt()

In [10]:
%%time
from tqdm import tqdm

tagged_corpus_list = []

for index, row in tqdm(df.iterrows(), total=len(df)):
    text = row["cleaned"]
    tag=row["title_lower"] # 소문자로 이루어진 제목
    tagged_corpus_list.append(TaggedDocument(tags=[tag], words=okt.morphs(text))) # 도서 각 줄거리에 제목이 tag로 포함된 data
    
print('문서의 수 :', len(tagged_corpus_list))

100%|█████████████████████████████████| 2382/2382 [00:06<00:00, 341.92it/s]

문서의 수 : 2382
CPU times: total: 8.23 s
Wall time: 6.97 s


In [11]:
from gensim.models import Doc2Vec
model2 = Doc2Vec(vector_size=300, workers=8, window=8)

In [12]:
# Vocabulary 빌드
model2.build_vocab(tagged_corpus_list)

# Doc2Vec 학습
model2.train(tagged_corpus_list, total_examples=model2.corpus_count, epochs=20)

In [14]:
title = "the Murder of Roger ackroyd"
model2.dv.most_similar(title) # 대문자가 들어가면서 에러값 

KeyError: "Key 'the Murder of Roger ackroyd' not present in vocabulary"

In [15]:
def book_recommend(title):
    try:
        for similar_doc in model2.dv.most_similar(title.lower()):
            # print(similar_doc) # 제목, 유사도
            print(df[df['title_lower']==similar_doc[0]].title.values[0]) # 제목만 추출
    except:
        print(f"입력하신 도서 '{title}'에 대한 정보를 찾을 수 없습니다.")

In [18]:
title = input('도서 제목 입력 : ') #the da vinci code
book_recommend(title)

도서 제목 입력 :  the Murder of Roger ackroyd


The Eyes of the Dragon
Equal Rites
Still Life
The Maltese Falcon
Crime and Punishment
Miss Peregrine's Home for Peculiar Children
The Girls
The Lightning Thief
The Woman in Cabin 10
Murder at the Vicarage


In [17]:
title = input('도서 제목 입력 : ') # The Da Vinci Code
book_recommend(title)

도서 제목 입력 :  The Da Vinci Code


Watership Down
The Girl with the Dragon Tattoo
The Story of Edgar Sawtelle
The Two Towers
Inferno
Origin
The Woman in White
Snow
A Discovery of Witches
Madame Bovary
